# Collision Avoidance - Data Collection（避障-数据收集）

如果你浏览了基本动作（Basic Motion）Notebook，希望你能享受到让 Jetbot 四处移动是多么的容易！太酷了！但更酷的是，让JetBot自己到处跑！

这是一个超难的任务，有许多不同的方法，但整个问题通常被分解成更容易的子问题。
可以说要解决的重要子问题，是防止机器人进入危险环境的问题！我们称之为``避免碰撞（Collision Avoidance）``。

在这组Notebook中，我们将尝试使用深度学习和一个非常通用的传感器（摄像头）来解决这个问题。你将看到如何用一个神经网络、相机、和NVIDIA Jetson Nano，我们可以教小车执行非常有用的行为！

避免碰撞的方法是在小车周围制造一个虚拟的``安全气泡（Safety bubble）``。在这个安全泡内，小车能够旋转成一个圆圈，而不会撞到任何物体（或其他危险情况，如从壁架上跌落）。
当然，小车受其视野的限制，我们不能阻止物体被放置在小车后面等等，但我们可以阻止小车进入这些场景。

我们的方法非常简单：
首先，我们将手动将小车放置在违反“安全泡沫”的场景中，并将这些场景标记为 ``blocked``，将小车看到的内容的快照与此标签（blocked）一起保存。

其次，我们将手动将小车放置在可以安全向前移动一点的场景中，并将这些场景标记为``free``。同样，我们将快照与此标签一起保存。

这就是我们在Notebook要做的一切：数据收集（Data Collection）。一旦我们有了大量的图像和标签，我们将把这些数据上传到一台支持GPU的机器上，在那里我们将训练一个神经网络，根据小车看到的图像来预测小车的安全气泡是否被破坏。最后，我们将使用它实现一个简单的避免碰撞行为：）

> 重要提示：当 JetBot 旋转到位时，它实际上是围绕两个轮子之间的中心旋转，而不是围绕机器人底盘本身的中心旋转。这是一个重要的细节要记住，当你试图估计是否违反了机器人的安全泡泡。但别担心，你不必说得很精确。如果有疑问的话，最好站在谨慎的一边（一个巨大的安全泡沫）。我们要确保JetBot不会进入一个无法通过原地转弯摆脱的场景。

### 显示摄像头的实时显示

我们开始吧。首先，让我们像在 **“遥感控制（teleoperation）"** 笔记本中那样初始化和显示相机。

> 由于使用的神经网络要求以 224x224 像素的图像作为输入。我们将把相机设置为这个尺寸，使数据集最小化（我们已经测试过它是否适用于此任务）
>
> 在某些情况下，最好以更大的图像大小收集数据，然后缩小到所需的大小。

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # 这个宽度和高度不一定要和相机匹配

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

太棒了，接下来让我们创建几个目录来存储所有数据。我们将创建一个文件夹“dataset”，其中包含两个子文件夹 ``free`` 和 ``blocked``，将每个场景的图像放到文件夹中。

In [ ]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# 使用 “try/expect"语句，如果目录已经存在，则下一个函数可能会引发错误。
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

刷新左侧的 Jupyter 文件浏览器，应该会看到这些目录出现了。接下来，让我们创建并显示一些用于保存每个类标签快照的按钮。我们还将添加一些文本框，这些文本框将显示到目前为止我们收集的每个类别的图像数量。这很有用，因为我们想确认所收集的``free``图像和``blocked``图像一样多。这也有助于了解我们总共收集了多少图像。

In [ ]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

现在，这些按钮还是没有功能的，必须附加函数来将每个类别的图像保存到按钮 ``on click`` 事件中。这里将保存``Image`` 小部件的数据而不是摄像头的，因为小部件的格式已经是压缩过的JPEG，这样会节省不少靠小。

为了确保不会重复命名（即使是在不同的机器上），我们使用python中的 ``uuid`` 包，它定义要生成的``uuid1`` 唯一标识符的方法。这个唯一标识符是根据当前时间和计算机地址等信息生成的。

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# 在附加回调部分，我们使用“lambda”函数忽略on click事件将提供给函数的参数，因为我们不需要它。
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

太棒啦！现在，上面的按钮应该将图像保存到 ``free`` 和 ``blocked`` 目录。您可以使用Jupyter实验室文件浏览器查看这些文件！
现在去收集一些数据

1. 将小车放置在被阻止的场景中，然后按 ``add blocked``
2. 将小车放置在自由的场景中，然后按 ``add free``
3. 重复1，2

>提醒：右键单击单元格并单击 ``Create New View for Output`` 可以将小部件移动到新窗口。或者我们将在最下重新一起显示

以下是一些标记数据的提示
1. 尝试不同的方向
2. 尝试不同的照明
3. 尝试各种对象/碰撞类型；墙、壁架、对象
4. 尝试不同的纹理地板/物体；有图案、光滑、玻璃等

最终，我们拥有的关于小车在现实世界中遇到的场景，数据越多则避碰行为就越好。这很重要

要获得各种各样的数据（如上述技巧所述），而不仅仅是大量的数据，但你可能需要每个类至少100张图片（这不是一门科学，这里只是一个有用的技巧）。但别担心，一旦你开始就会觉得很容易：）

In [ ]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

## 下一步（如果要在 Jetson Nano 上进行训练，可以忽略本步骤！）

一旦收集了足够的数据，可以将这些数据复制到我们的GPU桌面或云机器上进行训练。首先，我们可以调用下面的 *terminal* 命令来将数据集文件夹压缩放在一个 *zip* 文件中。

> ！前导符号表示要将单元格作为 *shell*（或 *terminal* ）命令运行。
> 下面zip命令中的-r 标志表示 *递归* 以便包含所有嵌套文件； -q标志表示 *安静* 以便zip命令不打印任何输出

In [ ]:
!zip -r -q dataset.zip dataset

应该会在Jupyter Lab文件浏览器中看到一个名为 ``dataset.zip`` 文件。可以使用Jupyter Lab文件浏览器通过右键单击并选择 ``download`` 将 zip文件下载的PC机。

接下来，我们需要将这些数据上传到我们的GPU桌面或云计算机器（我们称之为 *主机（host）*）来训练这个``避免碰撞`` 的神经网络。

我们假设你已经安排好了训练JetBot维基中描述的机器。如果有，可以导航到 ``http://<host_ip_address>：8888`` ，打开主机上运行的Jupyter Lab环境。

你需要打开的笔记本名为 ``collision_avoidance/train-model.ipynb`` 

所以去你的训练机，按照那里的指示去做！一旦你的模型被训练，我们将返回到机器人Jupyter Lab环境使用模型进行现场演示！